In [32]:
# processing the data sets
def processWord(tlString):
    import re

    match = re.findall("(?<=>).*?(?=<)", tlString)[0]

    if match=="":
        return None
       
    index = re.search('[a-zA-Z]', match)
    if index != None:
        return None

    # Removes parenthesis section
    match = re.sub("[\（\[].*?[\）\]]", "", match)

    # Removing other non-relevant characters that show up
    match = re.sub('[《》]', '', match)
        # match = match[:bracket1] + match[bracket1+1:bracket2] + match[bracket2+1:]

    colon = match.find("：")
    if colon != -1:
        match = match[:colon]
    
    match = re.sub('[…]', '', match)

    slash = match.find("／")
    if slash != -1:
        match = match[:slash]

    # Removing any commas or semicolons
    # We just want the first entry, which occurs before the commas/semicolons
    comma = match.find("，")
    if comma != -1:
        match = match[:comma]

    comma2 = match.find("、")
    if comma2 != -1:
        match = match[:comma2]

    semic = match.find("；")
    if semic != -1:
        match = match[:semic]

    semic2 = match.find("; ")
    if semic2 != -1:
        match = match[:semic2]

    return match


In [35]:
# BATCH 1

import requests
import json
from urllib.error import HTTPError
from bs4 import BeautifulSoup

# Load english text set
engWords = []
with open("C:/Users/jkami/OneDrive/Documents/UCSB/SPR23/LING111 Project/english_lexicon.txt") as in_file:
    for line in in_file:
        engWords.append(line.strip())

engChnDict = dict()

# Iterating over every word in our text set
i = 0
for word in engWords[:25000]:
    if (i % 1190)==0:
        print(i)
    i += 1
    # Creating URL to query
    wordUrl = f"https://dictionary.cambridge.org/dictionary/english-chinese-simplified/{word}"
    
    # Getting HTMl, formatting, finding translation segment
    try:
        r = requests.get(wordUrl, headers={"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) \
        AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"})
    except HTTPError as err:
        continue
    rhtml = BeautifulSoup(r.content, 'html.parser')
    tlStrings = rhtml.find_all(class_="trans dtrans dtrans-se break-cj", limit=3)

    # If word isn't in dictionary, rhtml.find_all returns []
    # as there is no trans dtrans dtrans-se break-cj class, since
    # Cambridge sends us to the homepage when a word isn't in the dictionary
    if tlStrings == []:
        continue
    else: 
        # Word is in dictionary, so get the translation
        tlLists = []
        for item in tlStrings:
            equiv = processWord(str(item))
            if equiv != '':
                tlLists.append(equiv)
                
    engChnDict[word] = tlLists

# Pushing the whole dictionary to a JSON file!
with open("cambridge dict jsons/cambridgedict three senses 1.json", "w", encoding='utf-8-sig') as out_file:
    json.dump(engChnDict, out_file, indent=4, ensure_ascii=False)


0
